In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from sklearn.utils import shuffle
from scipy.io.wavfile import read
import matplotlib.pyplot as plt
import os

In [9]:
path = '/Users/abylikhsanov1/AI/kaggle/speech_commands_v0.01/'
path_audio = '/Users/abylikhsanov1/AI/kaggle/speech_commands_v0.01/'
test_list = []
valid_list = []
with open(path+'testing_list.txt') as testing_file:
    data = testing_file.readlines()
    for line in data:
        test_list.append(line.strip())
        
with open(path+'validation_list.txt') as testing_file:
    data = testing_file.readlines()
    for line in data:
        valid_list.append(line.strip())
        


In [10]:
classes = ['yes','no','up','down','left','right','on','off','stop','go',
           '_background_noise_','bed','bird','cat','dog','eight',
                  'five','four','happy','house','marvin','nine','one',
                  'seven','sheila','six','three','tree','two','wow','zero']
all_dir = []

for i in classes:
    for l in os.listdir(path+i):
        all_dir.append(i+'/'+l)

for data1,data2 in zip(test_list,valid_list):
    all_dir.remove(data1)
    all_dir.remove(data2)

In [11]:
#import soundfile as sf
%matplotlib inline

classes = ['yes','no','up','down','left','right','on','off','stop','go']

unknown_classes = ['_background_noise_','bed','bird','cat','dog','eight',
                  'five','four','happy','house','marvin','nine','one',
                  'seven','sheila','six','three','tree','two','wow','zero']
X_train = []
y_train = []
X_test = []
y_test = []
X_valid = []
y_valid = []

for l in all_dir:
    check = l.split('/')[0]
    
    try:
        data = read(path+l)
        X_train.append(data)
        if check in classes:
            y_train.append(classes.index(check))
        else:
            y_train.append(10)
    except ValueError:
        continue

/Users/abylikhsanov1/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [14]:
print(all_dir[20])

yes/8a28231e_nohash_3.wav


In [4]:
# Normalise

In [5]:
# The CNN

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

# These functions will train the model and modify the weights and biases:
logits = Len(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer()
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
saver = tf.train.Saver()